## Loading Data

In [ ]:
from wbfm.utils.general.utils_behavior_annotation import BehaviorCodes
from wbfm.utils.traces.triggered_averages import FullDatasetTriggeredAverages
from wbfm.utils.traces.triggered_averages import TriggeredAverageIndices

In [ ]:
import pandas as pd
data = pd.read_hdf("quartiled_data_0602.h5")

#### Function to map our states to wbfm enum values

In [ ]:
def mapping(cls, flip: bool = False):
    original_mapping = {
        "forward": cls.FWD,
        "reversal": cls.REV,
        "sustained reversal": cls.REV,
        "ventral": cls.VENTRAL_TURN,
        "dorsal": cls.DORSAL_TURN,
        6: cls.SUPERCOIL,
        7: cls.QUIESCENCE,
        0: cls.NOT_ANNOTATED,
        -99: cls.UNKNOWN,  # Should not be in any files that Ulises produces
    }
    if flip:
        original_mapping = {v: k for k, v in original_mapping.items()}
    return original_mapping

#### Function to get a dataframe with the triggered averages for each of the dataset

In [ ]:
def get_triggered_averages(data, fixed_num_points_after_event=None):
    all_dfs = []
    groups = data.groupby("dataset")
    for name, df in groups:
        states = df['state'].map(mapping(BehaviorCodes))
        opt = dict(behavioral_annotation=states, fixed_num_points_after_event=fixed_num_points_after_event)
        indices = TriggeredAverageIndices(**opt)
        traces = FullDatasetTriggeredAverages(df.loc[:,~df.columns.isin(["state","dataset"])], indices)
        triggered_avg_df = traces.df_of_all_triggered_averages()
        triggered_avg_df['dataset'] = name
        triggered_states = states[states.index.isin(triggered_avg_df.index)]
        all_dfs.append(triggered_avg_df)
    return pd.concat(all_dfs)                         

In [ ]:
triggered_1203 = get_triggered_averages(data)
triggered_1303 = get_triggered_averages(data, 142) # 140 because 20200826_w6 doesn't have that many frames, there are some recordings with a reversal interval of way over 150. This is fine        

In [ ]:
triggered_1203.groupby('dataset').size().head()

In [ ]:
triggered_1303.groupby('dataset').size().head()


In [ ]:
triggered_1303.to_hdf("triggered_averages_1903.h5", key="triggered_averages")